<a href="https://colab.research.google.com/github/sakibreza/Emotion-Recognition-from-Multi-Channel-EEG-Signals/blob/main/sklearn_deap_ML_training_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
import pickle as pickle
import matplotlib.pyplot as plt
import os
import sys
import math
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
import sklearn
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from scipy.io import loadmat 
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
torch.manual_seed(13)
np.random.seed(7)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input_height = 9
input_width = 9
input_channel_num = 4
# conv_fuse = "plus"

# conv_1_shape = '4*4*1*16'
# pool_1_shape = 'None'

# # conv_2_shape = 'None'
# conv_2_shape = '4*4*1*32'
# pool_2_shape = 'None'

# # conv_3_shape = 'None'
# conv_3_shape = '4*4*1*64'
# pool_3_shape = 'None'

# conv_4_shape = '1*1*128*4'
# pool_4_shape = 'None'

time_step = 1
window_size = 1
# convolution full connected parameter
fc_size = 1024

dropout_prob = 0.5
np.random.seed(3)

calibration = 'N'
norm_type = '2D'
regularization_method = 'dropout'
enable_penalty = True

In [ ]:
arousal_key = "arousal_labels"
valence_key = "valence_labels"

# def minus(item):
#     return item-1

def construct_data(input_file, arousal_or_valence=None, bands=[0, 1, 2, 3]):
    input_channel_num = len(bands) * time_step

    dataset_dir = '/content/drive/MyDrive/DEAP_3D//DE3D_'
    ###load training set
    print("loading ",dataset_dir+input_file,".mat")
    data_file = sio.loadmat(dataset_dir+input_file+".mat")


    cnn_datasets = data_file["data"]

    #start
    arousal_labels = data_file[arousal_key]
    valence_labels = data_file[valence_key]

    arousal_labels = np.where(arousal_labels == 1, 2, arousal_labels)

    curr_labels = np.add(arousal_labels, valence_labels)

    # 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
    labels = curr_labels[0]

    # print('labels: ', np.unique(labels, return_counts= True))

    cnn_datasets = cnn_datasets.transpose(0,2,3,1)

    cnn_datasets = cnn_datasets[:,:,:,bands]

    cnn_datasets = cnn_datasets.reshape(len(cnn_datasets)//time_step, window_size,input_height,input_width,input_channel_num)

    index = np.array(range(0, len(labels)))
    np.random.shuffle(index)

    cnn_datasets   = cnn_datasets[index].reshape((-1, 4*9*9))
    labels  = labels[index]

    return cnn_datasets, labels  

In [ ]:
def multi_acc(y_test, y_pred):   
    acc = accuracy_score(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)
    return acc, conf_mat

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def report_result(best_model, data_X, data_y, label):
  print('\n',label,'\n----------------\n')
  nb_classes = 4
  data_y_hat = model.predict(data_X)
  data_acc, confusion_matrix = multi_acc(data_y, data_y_hat)
  print('****** mean classification accuracy: ', round(data_acc*100, 2), '% ******')
  # print(confusion_matrix)
  print('\n Accuracy per class\n ---------------')
  acc_per_class = np.diag(confusion_matrix)/np.sum(confusion_matrix, 1)

  for i in range(len(acc_per_class)):
    print(label_mapping[i], ': ', round(acc_per_class[i]*100, 2), '%')
  print('****** mean per class accuracy: ', round(np.mean(acc_per_class)*100, 2),'% ******\n')

In [ ]:
# 0 = LALV, 1 = LAHV, 2 = HALV, 3 = HAHV
label_mapping = {0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
def get_result(best_model, data_X, data_y, label):
  ret = {}
  nb_classes = 4
  data_y_hat = model.predict(data_X)
  data_acc, confusion_matrix = multi_acc(data_y_hat, data_y)
  ret['acc'] = round(data_acc*100, 2)
  acc_per_class = np.diag(confusion_matrix)/np.sum(confusion_matrix, 1)
  for i in range(len(acc_per_class)):
    ret[label_mapping[i]] = round(acc_per_class[i]*100, 2)
  ret['per_class_acc'] = round(np.mean(acc_per_class)*100, 2)

  return ret

In [ ]:
{0: 'LALV', 1: 'LAHV', 2: 'HALV', 3: 'HAHV'}
acc_list = []
pc_acc_list = []
lalv_acc_list = []
lahv_acc_list = []
halv_acc_list = []
hahv_acc_list = []

for i in range(16):
  sub_id = i + 1
  print("**************** Process starting for subject no. ", sub_id, "**************** \n")
  if sub_id < 10:
    input_file = 's0' + str(sub_id)
  else:
    input_file = 's' + str(sub_id)
  data, labels = construct_data(input_file)
  X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
  model = svm.SVC(C=10000, kernel='rbf')
  model.fit(X_train, y_train)
  ret = get_result(model, X_test, y_test, 'Test Data')
  print('accuracy: ', ret['acc'], '%')
  acc_list.append(ret['acc'])
  pc_acc_list.append(ret['per_class_acc'])
  lalv_acc_list.append(ret['LALV'])
  lahv_acc_list.append(ret['LAHV'])
  halv_acc_list.append(ret['HALV'])
  hahv_acc_list.append(ret['HAHV'])

**************** Process starting for subject no.  1 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s01 .mat
accuracy:  96.04 %
**************** Process starting for subject no.  2 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s02 .mat
accuracy:  81.67 %
**************** Process starting for subject no.  3 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s03 .mat
accuracy:  87.71 %
**************** Process starting for subject no.  4 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s04 .mat
accuracy:  87.29 %
**************** Process starting for subject no.  5 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s05 .mat
accuracy:  77.71 %
**************** Process starting for subject no.  6 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s06 .mat
accuracy:  92.92 %
**************** Process starting for subject no.  7 **************** 

loading  /content/drive/MyDrive/DEAP_3D//DE3D_s07 .m

In [ ]:
result_dict =   {'acc': acc_list,
  'pc_acc': pc_acc_list,
  'lalv': lalv_acc_list,
  'lahv': lahv_acc_list,
  'halv': halv_acc_list,
  'hahv': hahv_acc_list}

In [ ]:
np.mean(np.array(result_dict['acc']))

89.245

In [ ]:
result_dict

{'acc': [81.25,
  75.0,
  78.75,
  86.25,
  71.88,
  83.54,
  81.88,
  76.25,
  78.12,
  84.17,
  83.54,
  81.88,
  91.25,
  81.46,
  82.29,
  82.08],
 'hahv': [88.65,
  80.86,
  100.0,
  81.58,
  78.32,
  83.56,
  85.85,
  84.28,
  85.63,
  86.84,
  80.77,
  88.12,
  91.58,
  83.33,
  86.46,
  82.11],
 'halv': [80.83,
  70.83,
  69.51,
  86.05,
  69.84,
  94.0,
  80.77,
  79.31,
  73.83,
  88.79,
  86.76,
  80.56,
  95.71,
  89.44,
  80.0,
  86.18],
 'lahv': [81.01,
  71.88,
  81.54,
  95.65,
  70.2,
  87.37,
  81.2,
  74.07,
  77.05,
  84.0,
  88.17,
  82.61,
  75.0,
  71.79,
  86.26,
  82.05],
 'lalv': [74.29,
  69.37,
  77.17,
  87.45,
  67.48,
  68.6,
  73.75,
  65.87,
  73.1,
  78.36,
  78.42,
  68.0,
  80.77,
  74.14,
  77.34,
  79.35],
 'pc_acc': [81.2,
  73.23,
  82.05,
  87.68,
  71.46,
  83.39,
  80.39,
  75.88,
  77.4,
  84.5,
  83.53,
  79.82,
  85.77,
  79.68,
  82.52,
  82.42]}

In [ ]:
np.save('result_svm_linear_c.npy', result_dict)

In [ ]:
temp = np.load('result_svm_rbf.npy', allow_pickle=True)

In [ ]:
np.mean(np.array(temp.item()['acc']))

92.78625